In [2]:
import numpy as np
from time import perf_counter_ns

#метод прогонки
def triang_matrix_method(a,b,c,f):
    t1_start = perf_counter_ns()
    a, b, c, f = tuple(map(lambda k_list: list(map(float, k_list)), (a, b, c, f)))
    alpha = []
    beta = []
    
    alpha.insert(0, -b[0] / c[0])
    beta.insert(0, f[0] / c[0])
    n = len(f)
    x = [0]*n
    
    for i in range(1, n):
        alpha.insert(i, -b[i]/(a[i]*alpha[i-1] + c[i]))
        beta.insert(i, (f[i] - a[i]*beta[i-1])/(a[i]*alpha[i-1] + c[i]))
    
    x[n-1] = beta[n-1]

    for i in range(n-1, 0, -1):
        x[i - 1] = alpha[i - 1]*x[i] + beta[i - 1]
    
    t1_stop = perf_counter_ns()
    print(t1_stop - t1_start)
    
    return x



#main diagonal = c
#a lower
#b upper
a = [0, -1, -1]
b = [-1, -1, 0]
c = [2, 2, 2]
f = [2, 2, 2]

print(triang_matrix_method(a,b,c,f))


34000
[3.0, 3.9999999999999996, 2.9999999999999996]


In [3]:
#метод прогонки
def triang_matrix_method_np(a,b,c,f):
    t1_start = perf_counter_ns()
    #a, b, c, f = tuple(map(lambda k_list: list(map(float, k_list)), (a, b, c, f)))
    n = f.shape[0]
    alpha = np.asarray([0]*n, dtype=np.float)
    beta = np.asarray([0]*n, dtype=np.float)
    
    alpha[0] = -b[0] / c[0]
    beta[0] = f[0] / c[0]
    
    x = np.zeros(n)
    
    for i in range(1, n):
        alpha[i]= -b[i]/(a[i]*alpha[i-1] + c[i])
        beta[i] = (f[i] - a[i]*beta[i-1])/(a[i]*alpha[i-1] + c[i])
    
    x[n-1] = beta[n-1]

    for i in range(n-1, 0, -1):
        x[i - 1] = alpha[i - 1]*x[i] + beta[i - 1]
    
    t1_stop = perf_counter_ns()
    print(t1_stop - t1_start)
    
    return x

#main diagonal = c
#a lower
#b upper
a = np.array([0, -1, -1], dtype=np.float)
b = np.array([-1, -1, 0], dtype=np.float)
c = np.array([2, 2, 2], dtype=np.float)
f = np.array([2, 2, 2], dtype=np.float)

print(triang_matrix_method_np(a,b,c,f))

85100
[3. 4. 3.]


In [4]:
#получение матрицы L
def get_L(m):
    L = m.copy()
    for i in range(L.shape[0]):
            L[i, i] = 1
            L[i, i+1 :] = 0
    return np.matrix(L)

In [5]:
#получение матрицы U
def get_U(m):
    U = m.copy()
    for i in range(1, U.shape[0]):
        U[i, :i] = 0
    return U

In [6]:
#метод LU разложения

#разложение матрицы на коэфициенты
def decompose_to_LU(a):
    

    # create emtpy LU-matrix
    lu_matrix = np.matrix(np.zeros([a.shape[0], a.shape[1]]))
    n = a.shape[0]

    for k in range(n):
        for j in range(k, n):
            lu_matrix[k, j] = a[k, j] - lu_matrix[k, :k] * lu_matrix[:k, j]
        for i in range(k + 1, n):
            lu_matrix[i, k] = (a[i, k] - lu_matrix[i, :k] * lu_matrix[:k, k]) / lu_matrix[k, k]

    return lu_matrix


In [7]:
def solve_LU(lu_matrix, b):
    t2_start = perf_counter_ns()
    
    # get supporting vector y
    y = np.matrix(np.zeros([lu_matrix.shape[0], 1]))
    for i in range(y.shape[0]):
        y[i, 0] = b[i, 0] - lu_matrix[i, :i] * y[:i]

    # get vector of answers x
    x = np.matrix(np.zeros([lu_matrix.shape[0], 1]))
    for i in range(1, x.shape[0] + 1):
        x[-i, 0] = (y[-i] - lu_matrix[-i, -i:] * x[-i:, 0] )/ lu_matrix[-i, -i]
        
    t2_stop = perf_counter_ns()
    print(t2_stop - t2_start)

    return x

In [10]:
matrix = np.array([[1, 1/2, 1/3], [1/3, 3/4, 1/5], [1/4, 1/5, 1/2]], dtype=np.float)
LU = decompose_to_LU(matrix)
L = get_L(LU)
U = get_U(LU)

print(matrix)
print(L)
print(U)
print(LU)
print(L*U)

print(solve_LU(LU, np.transpose(np.array([[2, 2, 2]], dtype=np.float))))

[[1.         0.5        0.33333333]
 [0.33333333 0.75       0.2       ]
 [0.25       0.2        0.5       ]]
[[1.         0.         0.        ]
 [0.33333333 1.         0.        ]
 [0.25       0.12857143 1.        ]]
[[1.         0.5        0.33333333]
 [0.         0.58333333 0.08888889]
 [0.         0.         0.4052381 ]]
[[1.         0.5        0.33333333]
 [0.33333333 0.58333333 0.08888889]
 [0.25       0.12857143 0.4052381 ]]
[[1.         0.5        0.33333333]
 [0.33333333 0.75       0.2       ]
 [0.25       0.2        0.5       ]]
688200
[[0.01410106]
 [1.78613396]
 [3.27849589]]
